In [266]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression

import warnings

warnings.filterwarnings('ignore')


In [267]:
df = pd.read_csv('heart_disease.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [268]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [269]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [270]:
df.shape

(303, 14)

In [271]:
df.fillna(df.median(), inplace=True)


In [272]:
df.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [273]:
df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [274]:
X = df.drop('target', axis=1)
y = df['target']

In [275]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [276]:
X.shape

(303, 13)

In [277]:
print("Scaler was fitted with:", scaler.n_features_in_, "features")




Scaler was fitted with: 13 features


In [278]:
df["target"].value_counts()

target
1    165
0    138
Name: count, dtype: int64

In [279]:
df_majority = df[df.target == 1]
df_minority = df[df.target == 0]

data_minority_upsampled = resample(df_minority, replace=True, n_samples = 165, random_state = 42)
df_majority.shape, df_minority.shape



((165, 14), (138, 14))

In [280]:
data_minority_upsampled.shape


(165, 14)

In [281]:
df = pd.concat([df_majority, data_minority_upsampled])


In [282]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [283]:
models = {
    'Logistic Regression': LogisticRegression(),
    'SVM': SVC(),
    'Decision Tree': DecisionTreeClassifier()
}

param_grid = {
    'Logistic Regression': {'C': [0.1, 1, 10]},
    'SVM': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
    'Decision Tree': {'max_depth': [3, 5, 10]}
}

best_models = {}

for model_name, model in models.items():
    grid_search = GridSearchCV(model, param_grid[model_name], cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    best_models[model_name] = best_model

    y_pred = best_model.predict(X_test)

    print(f"{model_name} - Best Params: {grid_search.best_params_}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")


Logistic Regression - Best Params: {'C': 10}
Accuracy: 0.8525
Precision: 0.8710
Recall: 0.8438
SVM - Best Params: {'C': 1, 'kernel': 'rbf'}
Accuracy: 0.8689
Precision: 0.9000
Recall: 0.8438
Decision Tree - Best Params: {'max_depth': 3}
Accuracy: 0.8197
Precision: 0.8387
Recall: 0.8125


1 → "Heart Disease"
0 → "No Heart Disease"

In [284]:
print("Scaler was fitted on:", scaler.n_features_in_)


Scaler was fitted on: 13


In [285]:
best_model = best_models['Logistic Regression']

Input_Data = (56,1,1,120,236,0,1,178,0,0.8,2,0,2)  # Tuple input
Input_data_as_numpy_array = np.asarray(Input_Data).reshape(1, -1)  # Convert to 2D array
std_data = scaler.transform(Input_data_as_numpy_array)  # Standardize input
Prediction = best_model.predict(std_data)  # Make prediction

if Prediction[0] == 0:
    print("The person does NOT have heart disease.")
else:
    print("The person HAS heart disease.")

The person HAS heart disease.


In [286]:
import pickle 
filename = 'heart_disease.pkl'
with open(filename, 'wb') as file:
    pickle.dump(best_model, file)

print(f'Model has been saved as {filename}')


Model has been saved as heart_disease.pkl


In [287]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

print("New scaler.pkl saved with 13 features.")

New scaler.pkl saved with 13 features.


In [288]:
# Ensure input data has the correct number of features (13 in this case)
Input_Data = (56, 1, 1, 120, 236, 0, 1, 178, 0, 0.8, 2, 0, 2)  # Tuple input

# Convert input data to a numpy array and reshape it
Input_data_as_numpy_array = np.asarray(Input_Data).reshape(1, -1)  # Convert to 2D array

# Check the number of features in the input data
print("Input data shape:", Input_data_as_numpy_array.shape)  # Should be (1, 13)

# Standardize the input data using the same scaler used during training
std_data = scaler.transform(Input_data_as_numpy_array)  # Standardize input

# Make prediction using the best model
Prediction = best_model.predict(std_data)  # Make prediction

# Display the result
if Prediction[0] == 0:
    print("The person does NOT have heart disease.")
else:
    print("The person HAS heart disease.")

Input data shape: (1, 13)
The person HAS heart disease.


In [289]:
with open('heart_disease.pkl', 'rb') as file: # read binary mode 0/1
    loaded_model = pickle.load(file)


new_prediction = loaded_model.predict(X_test)
print('Prediction from loaded model:', new_prediction)

print(f'The List contains {len(new_prediction)} predictions')
print(X_test.shape)

Prediction from loaded model: [0 1 1 0 1 1 1 0 0 0 1 0 1 0 1 1 1 0 0 0 1 0 0 1 1 1 1 1 0 1 0 0 0 0 1 0 1
 1 1 1 0 1 1 1 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0]
The List contains 61 predictions
(61, 13)
